In [2]:
!pip install wandb pandas transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 150.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 159.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 134.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 163.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 81.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, random_split
import wandb
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import time
import torch
from torch.utils.data import Dataset

In [4]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import files
uploaded = files.upload()

In [5]:
df = pd.read_csv("combined.csv")

In [6]:
df

,Game Number,Move Number,Player,Move,Board,Comment
0,576,1,White,b2b4,r n b q k b n r\np p p p p p p p\n. . . . . . ...,"WHAT?!?! In the true hypermodern style, Tartak..."
1,576,2,Black,e7e6,r n b q k b n r\np p p p p p p p\n. . . . . . ...,Maroczy (Black) makes an illogical move of his...
2,576,3,White,c1b2,r n b q k b n r\np p p p . p p p\n. . . . p . ...,Tartakower doesn't need to defend the pawn yet...
3,576,4,Black,g8f6,r n b q k b n r\np p p p . p p p\n. . . . p . ...,"Maroczy follows more conventional lines, devel..."
4,576,5,White,b4b5,r n b q k b . r\np p p p . p p p\n. . . . p n ...,Tartakower inhibits the development of Black's...
...,...,...,...,...,...,...
49495,3,94,Black,h5h3,. . . . . . . .\n. . . . k . p .\n. p n . . p ...,NaN
49496,3,95,White,b1b6,. . . . . . . .\n. . . . k . p .\n. p n . . p ...,There goes the extra pawn.
49497,3,96,Black,e7d6,. . . . . . . .\n. . . . k . p .\n. R n . . p ...,NaN
49498,3,97,White,g1g2,. . . . . . . .\n. . . . . . p .\n. R n k . p ...,NaN


In [7]:
df = df.dropna().reset_index()

In [8]:
import re

def board_to_fen(board, player, move_number):
    # Extract rows and reverse to match FEN order (from 8th to 1st rank)
    rows = board.strip().split('\n')

    # Convert board rows to FEN format
    fen_rows = []
    for row in rows:
        clean_row = re.sub(r"\s+", "", row)  # Remove spaces
        # Correctly replace consecutive dots with the count
        fen_row = re.sub(r"(\.+) ", lambda m: str(len(m.group(1))), clean_row)  # Incorrectly placed space
        fen_row = re.sub(r"\.+", lambda m: str(len(m.group(0))), fen_row)  # Handle all dots
        fen_rows.append(fen_row)

    fen_pieces = "/".join(fen_rows)

    # Active color ('w' for White, 'b' for Black)
    active_color = 'w' if player == 'White' else 'b'

    # Default values for other FEN components (assuming basic capabilities)
    castling_availability = 'KQkq'  # Assuming all castling is still possible
    en_passant_target = '-'  # No en passant target
    halfmove_clock = '0'  # Reset on pawn moves or captures, not shown here
    fullmove_number = str(move_number)

    # Compile full FEN string
    fen = f"{fen_pieces} {active_color} {castling_availability} {en_passant_target} {halfmove_clock} {fullmove_number}"
    return fen

# Assuming df is your DataFrame
df['FEN'] = df.apply(lambda row: board_to_fen(row['Board'], row['Player'], row['Move Number']), axis=1)
df.head()


,index,Game Number,Move Number,Player,Move,Board,Comment,FEN
0,0,576,1,White,b2b4,r n b q k b n r\np p p p p p p p\n. . . . . . ...,"WHAT?!?! In the true hypermodern style, Tartak...",rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...
1,1,576,2,Black,e7e6,r n b q k b n r\np p p p p p p p\n. . . . . . ...,Maroczy (Black) makes an illogical move of his...,rnbqkbnr/pppppppp/8/8/1P6/8/P1PPPPPP/RNBQKBNR ...
2,2,576,3,White,c1b2,r n b q k b n r\np p p p . p p p\n. . . . p . ...,Tartakower doesn't need to defend the pawn yet...,rnbqkbnr/pppp1ppp/4p3/8/1P6/8/P1PPPPPP/RNBQKBN...
3,3,576,4,Black,g8f6,r n b q k b n r\np p p p . p p p\n. . . . p . ...,"Maroczy follows more conventional lines, devel...",rnbqkbnr/pppp1ppp/4p3/8/1P6/8/PBPPPPPP/RN1QKBN...
4,4,576,5,White,b4b5,r n b q k b . r\np p p p . p p p\n. . . . p n ...,Tartakower inhibits the development of Black's...,rnbqkb1r/pppp1ppp/4pn2/8/1P6/8/PBPPPPPP/RN1QKB...


In [9]:
config = {
    'Model': 'gpt2',
    'NUMER_OF_DATA_DIRS': 12,
    'batch_size': 2,
    'lr':  3e-5,
    'train_precentege': 0.9,
    'epochs': 5,
    'data_to_use': {'<fen>': True, '<moves>': True, '<last move description>': False,
                    '<legal moves>': False, '<attacked by>': False, '<attacks>': False},
    'max_length':300
}

In [10]:
# combined csv
# def convert_data_to_text(row, max_length=config['max_length'], end_of_text_token=""):
#     # Assuming 'FEN', 'moves', and 'comment' are column names in the DataFrame
#     FEN, moves, comment = row['FEN'], row['Move'], row['Comment']
#     FEN, moves, comment = FEN[:max_length], moves[:max_length], comment[:max_length]
#     token_to_data = {'<fen>': FEN, '<moves>': moves}
#     text = ""
#     for token, data in token_to_data.items():
#         text += f"{token} {data} "
#     text += f"<comment> {comment} {end_of_text_token}"
#     return text

def convert_data_to_text(row, max_length=config['max_length'], end_of_text_token=""):
    # Assuming 'FEN', 'moves', and 'comment' are column names in the DataFrame
    FEN, moves, comment = row['FEN'], row['Move'], row['Comment']
    FEN, moves, comment = FEN[:max_length], moves[:max_length], comment[:max_length]

    # Split moves into a list
    move_list = moves.split(',')

    # Truncate move history to last 5 moves (or less for initial moves)
    history_moves = move_list[-min(len(move_list), 5):]  # Get last 5 or less moves
    history_moves_str = ", ".join(history_moves)  # Join moves with comma separator

    token_to_data = {'<fen>': FEN, '<moves>': history_moves_str}
    text = ""
    for token, data in token_to_data.items():
        text += f"{token} {data} "
    text += f"<comment> {comment} {end_of_text_token}"
    return text


# def convert_data_to_text_2(data_object, max_length=768, end_of_text_token="<|endoftext|>"):
#     (FEN, moves, last_move_desc, legal_moves, attackers_list, attacks_list, comment) = data_object
#     (FEN, moves, last_move_desc, legal_moves,
#      attackers_list, attacks_list, comment) = (FEN[:max_length], moves[:max_length], last_move_desc[:max_length],
#                                                legal_moves[:max_length], attackers_list[:max_length],
#                                                attacks_list[:max_length], comment[:max_length])
#     moves = moves.split(',')[-1].strip() if ',' in moves else moves

#     token_to_data = {'<fen>': FEN, '<moves>': moves, '<last move description>': last_move_desc,
#                      '<legal moves>': legal_moves, '<attacked by>': attackers_list, '<attacks>': attacks_list}
#     text = ""
#     for token in token_to_data.keys():
#         if config['data_to_use'][token]:
#             text += f"{token} {token_to_data[token]} "
#     text += f"<comment> {comment} {end_of_text_token}"  # comment always included at the end + end token

#     return text

def convert_data_to_text_2(data_object, max_length=768, end_of_text_token="<|endoftext|>"):
    (FEN, moves, last_move_desc, legal_moves, attackers_list, attacks_list, comment) = data_object
    (FEN, moves, last_move_desc, legal_moves,
     attackers_list, attacks_list, comment) = (FEN[:max_length], moves[:max_length], last_move_desc[:max_length],
                                               legal_moves[:max_length], attackers_list[:max_length],
                                               attacks_list[:max_length], comment[:max_length])
    moves = moves.split(',')[-1].strip() if ',' in moves else moves

    # Split moves into a list
    move_list = moves.split(',')

    # Truncate move history to last 5 moves (or less for initial moves)
    history_moves = move_list[-min(len(move_list), 5):]  # Get last 5 or less moves
    history_moves_str = ", ".join(history_moves)  # Join moves with comma separator

    token_to_data = {'<fen>': FEN, '<moves>': history_moves_str, '<last move description>': last_move_desc,
                     '<legal moves>': legal_moves, '<attacked by>': attackers_list, '<attacks>': attacks_list}
    text = ""
    for token in token_to_data.keys():
        if config['data_to_use'][token]:
            text += f"{token} {token_to_data[token]} "
    text += f"<comment> {comment} {end_of_text_token}"  # comment always included at the end + end token

    return text



In [11]:
import pickle

dataset_tokens = list(config['data_to_use'].keys()) + ['<comment>']


class ProcessDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=config['max_length']):
        self.tokenizer = tokenizer
        self.comment_encoding = tokenizer.get_added_vocab()['<comment>']
        self.proccessed_data = []
        self.attn_masks = []
        self.labels = []

        for _, row in df.iterrows():
            text = convert_data_to_text(row, max_length)
            enc_text = tokenizer(text, truncation=True, max_length=max_length, padding="max_length")
            inputs = enc_text['input_ids']
            label_idx = inputs.index(self.comment_encoding) + 1
            labels = [-100] * label_idx + inputs[label_idx:]

            self.proccessed_data.append(torch.tensor(inputs))
            self.attn_masks.append(torch.tensor(enc_text['attention_mask']))
            self.labels.append(torch.tensor(labels))

    def __len__(self):
        return len(self.proccessed_data)

    def __getitem__(self, index):
        return self.proccessed_data[index], self.attn_masks[index], self.labels[index]

class MovesDataset(Dataset):
    def __init__(self, paths, tokenizer, max_length=768):

        self.comment_encoding = tokenizer.get_added_vocab()['<comment>']

        self.proccessed_data = []
        self.attn_masks = []
        self.labels = []


        for path in paths:
            with open(path, 'rb') as file:
                raw_data = pickle.load(file)
            for data_object in raw_data:
                text = convert_data_to_text_2(data_object)

                enc_text = tokenizer(text, truncation=True, max_length=max_length, padding="max_length")

                inputs = enc_text['input_ids']
                label_idx = inputs.index(self.comment_encoding) + 1
                labels = [-100] * label_idx + inputs[label_idx:]

                self.proccessed_data.append(torch.tensor(inputs))
                self.attn_masks.append(torch.tensor(enc_text['attention_mask']))
                self.labels.append(torch.tensor(labels))

    def __len__(self):
        return len(self.proccessed_data)

    def __getitem__(self, index):
        return self.proccessed_data[index], self.attn_masks[index], self.labels[index]


In [12]:
import torch
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer

GPT2_TYPE = "gpt2"

class GPT2:
    def __init__(self):
        print("Initialization\n")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.tokenizer = GPT2Tokenizer.from_pretrained(GPT2_TYPE)
        special_tokens_dict = {
            'pad_token': '[PAD]',
            'additional_special_tokens': dataset_tokens
        }
        self.tokenizer.add_special_tokens(special_tokens_dict)
        # self.tokenizer.add_tokens(get_chess_tokens())

        self.configuration = GPT2Config.from_pretrained(GPT2_TYPE)
        self.model = GPT2LMHeadModel(self.configuration)

        # Resize token embeddings to accommodate new tokens
        self.model.resize_token_embeddings(len(self.tokenizer))

        self.model.to(self.device)

    def load_model(self, model_path):
        print("Loading model\n")
        state_dict = torch.load(model_path, map_location=self.device)
        self.model.load_state_dict(state_dict)
        self.model.to(self.device)


In [13]:
gpt2 = GPT2()
model = gpt2.model.train()
tokenizer = gpt2.tokenizer
max_length = config['max_length']
eof = '<|endoftext|>'

Initialization



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [14]:
games_data_path = '/home'
saved_models_path = '/home/models/'

In [ ]:
df["Comment"] = df["Comment"].fillna('No Comment')

In [15]:
dataset = ProcessDataset(df,tokenizer)
# dataset = MovesDataset([f'{games_data_path}1.p'], tokenizer, max_length=max_length)


In [16]:
train_size = int(config['train_precentege'] * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)


In [17]:
run = wandb.init(project="LmChess", config={'batch size': config['batch_size'], 'lr': config['lr'], 'epochs': config['epochs']})


wandb: Currently logged in as: mzkingsl6. Use `wandb login --relogin` to force relogin


In [ ]:
#validation

In [18]:

validation_proccessed_data, validation_attn_masks, validation_labels = next(iter(test_dataloader))

validation_input_encodings = []
for i in range(config['batch_size']):
  textual_validation_data = tokenizer.decode(token_ids = validation_proccessed_data[i], skip_special_tokens=False).split('<comment>')

  validation_target_text = textual_validation_data[1].split(eof)[0]
  validation_input_text = textual_validation_data[0]

  wandb.log({f"validation_target_text {i}": wandb.Html(f'<p>{validation_target_text}</p>')})
  wandb.log({f"validation_input_text {i}": wandb.Html(f'<p>{validation_input_text}</p>')})

  comment_idx = list(validation_proccessed_data[i]).index(dataset.comment_encoding) + 1
  validation_input_encoding = validation_proccessed_data[i][:comment_idx].unsqueeze(0).cuda()
  #validation_input_encoding  = tokenizer.encode(validation_input_text, return_tensors="pt").cuda()

  validation_input_encodings.append(validation_input_encoding)

In [20]:

optimizer = AdamW(model.parameters(), lr= config['lr'])
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=5000, num_training_steps=-1
)

loss = 0
pad_token_id = tokenizer('[PAD]')['input_ids'][0]

epochs = config['epochs']

for epoch in range(epochs):
    with tqdm(total=len(dataset) / 2) as pbar:
        for idx,entry in enumerate(train_dataloader):

            if idx % 500 == 0 and idx != 0:
              for i in range(config['batch_size']):
                with torch.no_grad():
                    outputs = model.generate(validation_input_encodings[i], num_beams=2, no_repeat_ngram_size=2, max_length=max_length+1, pad_token_id=pad_token_id)
                    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                wandb.log({f"output_text {i}": wandb.Html(f'<p>{output_text}</p>')})

            if idx % 1000 == 0:
              torch.save(model.state_dict(), f'{saved_models_path}{idx}_{time.time()}_{int(loss)}.bin')

            model.zero_grad()

            inputs = entry[0].cuda()
            attn_masks = entry[1].cuda()
            labels = entry[2].cuda()
            outputs = model(inputs, labels=labels, attention_mask = attn_masks)

            loss = outputs['loss']
            loss.backward()
            optimizer.step()
            scheduler.step()

            wandb.log({"epoch": epoch, "loss": loss})
            pbar.update(2)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
8216it [03:08, 43.63it/s]                            
8216it [03:05, 44.29it/s]                            
8216it [03:08, 43.64it/s]                            
8216it [03:05, 44.20it/s]                            
8216it [03:05, 44.37it/s]                            


In [21]:
torch.save(model.state_dict(), f'{saved_models_path}final.bin')

test

In [22]:
model_path = '/home/models/final.bin'

In [23]:
gpt2_test = GPT2()

gpt2_test.load_model(model_path)

gpt2_test.model = gpt2_test.model.eval().cuda()


Initialization

Loading model



In [24]:
tested_model = gpt2_test
max = config['max_length']
eof = '<|endoftext|>'
pad_token_id = tested_model.tokenizer('[PAD]')['input_ids'][0]

In [25]:

dataloader = test_dataloader
def get_results():
    # Set device based on availability of CUDA
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the model to the appropriate device
    tested_model.model.to(device)

    # Retrieve batch data
    processed_data, attn_masks, labels = next(iter(dataloader))

    # Initialize input encodings container
    input_encodings = []

    for i in range(config['batch_size']):
        # Decode the token_ids to text and split at <comment>
        textual_data = tested_model.tokenizer.decode(token_ids=processed_data[i], skip_special_tokens=False).split('<comment>')

        # Target text ends at EOF (assuming EOF is defined elsewhere in your code)
        target_text = textual_data[1].split(eof)[0]
        input_text = textual_data[0]

        # Find the index of <comment> and prepare input encoding
        comment_idx = list(processed_data[i]).index(tested_model.tokenizer.get_added_vocab()['<comment>']) + 1
        input_encoding = processed_data[i][:comment_idx].unsqueeze(0).to(device)

        input_encodings.append(input_encoding)

    # Initialize results container
    results = []

    for input_encoding in input_encodings:
        with torch.no_grad():
            # Generate output using the model's generate function
            outputs = tested_model.model.generate(
                input_encoding,
                num_beams=2,
                no_repeat_ngram_size=2,
                max_length=max + 1,  # assuming 'max' is defined as the maximum length
                pad_token_id=pad_token_id  # assuming 'pad_token_id' is defined elsewhere
            )
            output_text = tested_model.tokenizer.decode(outputs[0], skip_special_tokens=True)
            results.append(output_text)

    return results


In [26]:
results = get_results()
results

[" r1bqkb1r/5ppp/p1np1N2/4p3/1pP1P3/N7/PP3PPP/R1BQKB1R b KQkq - 0 20  d8f6  This is a-pawn.. The Knight on d5, but he can't be a pawn, the White's\nthe Black's position. Now it's pawn on e5-e5.Ng6, so he has been a5 and the b-file, and d4,\na5   without d-f5 is\nto be to the c-Bishop, he would have a\ne4. But, for the Black Rook on the board, without White King. He's without Black King's Queen's Rooks on f-d5 (a6-Rook to without f3-g5 to a Rf3, a Knight.Rg4-b5 on without the d6. If he's a piece,, it can be without e-c5).  Black Knight's King to play...Nf8-Knight. In the without without c3 and f4 and without pawns on c5...Nc3. This, which can play on g-h5) and it without g5 without b3 is the f6 and Black pawn. Black Queen.",
 " 2rq2rk/3bbp1p/p1p1pp2/3p4/4PP2/5B2/PPP1N1PP/R2Q1R1K w KQkq - 0 33  f4f5  This is a pawn, but it's pawns on the Black's Queen.. The White's a-pawn, the b5, so he can't be a5-e5 and the c-file, and d5.Ng6-f4,\nthe\ne4-Bishop, he has the board. Now the White Rook on

evaluation

In [59]:
from tqdm import tqdm
import torch
from datasets import load_metric
import tensorflow as tf
from nltk.translate.bleu_score import sentence_bleu


def perplexity(model, dataloader):
    eval_loss = 0
    with tqdm(total=len(dataloader)) as pbar:
        for idx, entry in enumerate(dataloader):
            with torch.no_grad():
                inputs = entry[0].cuda()
                attn_masks = entry[1].cuda()
                labels = entry[2].cuda()
                outputs = model(inputs, labels=labels, attention_mask=attn_masks)
            loss = outputs[0]
            eval_loss += loss.mean().item()
            pbar.update(2)
    final_eval_loss = eval_loss / len(dataloader)
    perplexity = torch.exp(torch.tensor(final_eval_loss))
    return perplexity


def bleurt(target_texts, output_texts):
    metric = load_metric("bleurt")
    tf.compat.v1.flags.DEFINE_string('f', '', '')

    scores = metric.compute(predictions=output_texts, references=target_texts)['scores']
    return scores


def bleu(target_texts, output_texts):
    scores = []
    for idx in range(len(output_texts)):
        reference = [target_texts[idx].split()]
        candidate = output_texts[idx].split()
        scores.append(sentence_bleu(reference, candidate))
    return scores

ModuleNotFoundError: No module named 'datasets'

In [ ]:
test_perplexity = perplexity(tested_model.model, dataloader)
print(test_perplexity)

3000it [01:31, 32.77it/s]

tensor(1.8895)


In [ ]:
import torch
from tqdm import tqdm

def get_targets_and_outputs(model, dataset, comment_encoding, pad_token_id, max_length=768, eof='<|endoftext|>'):
    target_texts = []
    output_texts = []
    with tqdm(total=len(dataset)) as pbar:
        for idx, entry in enumerate(dataset):

          textual_data = model.tokenizer.decode(token_ids=entry[0], skip_special_tokens=False)
          textual_data = textual_data.split('<comment>')[1].split(eof)[0]
          target_texts.append(textual_data)

          comment_idx = list(entry[0]).index(comment_encoding) + 1
          input_encoding = entry[0][:comment_idx].unsqueeze(0).cuda()
          with torch.no_grad():
              outputs = model.model.generate(input_encoding, num_beams=2, no_repeat_ngram_size=2, max_length=max_length+1, pad_token_id=pad_token_id)
              output_text = model.tokenizer.decode(outputs[0], skip_special_tokens=False)
              output_text = output_text.split('<comment>')[1].split(eof)[0]
          output_texts.append(output_text)

          pbar.update(1)
    return target_texts, output_texts

In [ ]:
target_texts, output_texts = get_targets_and_outputs(tested_model, dataset, dataset.comment_encoding, pad_token_id, max_length=max, eof=eof)

100%|██████████| 30000/30000 [1:15:52<00:00,  6.59it/s]


In [ ]:
test_bleurt = bleurt(target_texts, output_texts)

<ipython-input-143-c836eee1c833>:26: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("bleurt")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleurt contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleurt/bleurt.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ImportError: To be able to use bleurt, you need to install the following dependency: bleurt.
Please install it using 'pip install git+https://github.com/google-research/bleurt.git' for instance.

In [ ]:
print(sum(test_bleurt)/len(test_bleurt))

In [ ]:
test_bleu = bleu(target_texts, output_texts)

In [ ]:
print(sum(test_bleu)/len(test_bleu))